In [ ]:
import os

import panel as pn
import requests

from config import NOAA_RAW_ROOT, NOAA_SOURCE_ROOT, STATION

pn.extension("perspective")
pn.config.sizing_mode = "stretch_width"
PERSPECTIVE_THEME = "material-dark"

 ## Ingest NOAA Weather Observations

 Source: https://www1.ncdc.noaa.gov/pub/data/noaa/isd-lite/

 Source Description: https://www1.ncdc.noaa.gov/pub/data/noaa/isd-lite/isd-lite-format.txt

In [ ]:
STATIONS = (STATION,)
YEARS = range(2021, 1979, -1)

In [ ]:
def get_source_url(year, station):
    file = f"{station}-{year}.gz"
    url = f"{NOAA_SOURCE_ROOT}{year}/{file}"
    return url


source_url = get_source_url(2021, "722430-12960")
source_url

In [ ]:
def to_target_filename(source_url):
    parts = source_url.split("/")
    return NOAA_RAW_ROOT + "/" + parts[-2] + "/" + parts[-1]


target_filename = to_target_filename(source_url)
target_filename

In [ ]:
def ingest(source_url, target_file_name):
    if os.path.exists(target_filename):
        return "skipped"

    if not os.path.exists(os.path.dirname(target_filename)):
        try:
            os.makedirs(os.path.dirname(target_filename))
        except OSError as exc:  # Guard against race condition
            pass
            # if exc.errno != errno.EEXIST:
            # raise
    r = requests.get(source_url, allow_redirects=True)
    if r.status_code == 200:
        open(target_filename, "wb").write(r.content)
    return str(r.status_code)


ingest(source_url, target_filename)

In [ ]:
source_urls = []


for year in YEARS:
    for station in STATIONS:
        source_urls.append(get_source_url(year, station))

source_urls[0:3]

In [ ]:
results = []
for source_url in source_urls:
    target_filename = to_target_filename(source_url)
    result = to_target_filename(source_url)
    result = ingest(source_url, target_filename)
    results.append((target_filename, result))
results[0:3]

Previous: [Overview](overview.ipynb)
Next: [Ingest Ercot Loads](ingest_ercot_loads.ipynb)